In [24]:
import nltk
import pandas as pd 
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
lemma = WordNetLemmatizer()
lb = LabelEncoder()
import warnings
warnings.filterwarnings('ignore') 
from collections import Counter 
from string import punctuation 
from sklearn.metrics import precision_score, recall_score , f1_score
from gensim.models import Word2Vec
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential 
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense , Embedding , Bidirectional , LSTM , Dropout
from tensorflow.keras.optimizers import Adam
from tqdm.auto import tqdm

In [2]:
data = pd.read_csv("dataset.csv")
data = data.head(30000)
print(data.shape)

(30000, 3)


In [3]:
data.dropna(inplace=True)
data['Text'] = data['Text'].fillna('')
data.isna().sum()

Text        0
Language    0
Label       0
dtype: int64

In [4]:
data = data[data['Language'] == 'en']

string_to_remove = 'litigious'
data = data[~data['Label'].str.contains(string_to_remove)]

print(data)

                                                    Text Language        Label
3      Rwanda is set to host the headquarters of Unit...       en     positive
5      It sucks for me since I'm focused on the natur...       en     negative
7      @ShawnTarloff @itsmieu you can also relate thi...       en  uncertainty
8      Social Security. Constant political crises dis...       en     negative
9      @FilmThePoliceLA A broken rib can puncture a l...       en     negative
...                                                  ...      ...          ...
29995  Okay. We are getting somewhere. https://t.co/Z...       en  uncertainty
29996  Y’all don’t understand how excited I am! 🌺 htt...       en     positive
29997  Talking to each other.\n\nI'm am saddened toda...       en     negative
29998  @mushka_mushk @lisamwake Crashing parties not ...       en     positive
29999  @theogould_art @tonipayne One of the most beau...       en     positive

[22134 rows x 3 columns]


In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(Text):
    Text = Text.lower()
    Text = re.sub(r'[^a-zA-Z\s]', '', Text)
    tokens = nltk.word_tokenize(Text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)
data['Text'] = data['Text'].apply(preprocess_text)

In [6]:
x= data['Text']
y= data['Label']

In [7]:
train_texts, test_texts, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Initialize VADER
vader = SentimentIntensityAnalyzer()

# Predict labels for validation set
predicted_labels_val = val_texts.apply(lambda text: 'positive' if vader.polarity_scores(text)['compound'] >= 0.05
                                             else 'negative' if vader.polarity_scores(text)['compound'] <= -0.05
                                             else 'neutral')

# Predict labels for test set
predicted_labels_test = test_texts.apply(lambda text: 'positive' if vader.polarity_scores(text)['compound'] >= 0.05
                                             else 'negative' if vader.polarity_scores(text)['compound'] <= -0.05
                                             else 'neutral')

# Calculate validation accuracy
val_1 = accuracy_score(val_labels, predicted_labels_val)

# Calculate test accuracy
test_1 = accuracy_score(test_labels, predicted_labels_test)

print("Validation Accuracy:", val_1)
print("Test Accuracy:", test_1)

Validation Accuracy: 0.5367024280067758
Test Accuracy: 0.5412243053986898


In [8]:
label_mapping = {'positive': 1, 'negative': 0, 'uncertainty': 2}
data['Label'] = data['Label'].map(label_mapping)

# Split data into training and validation sets
x = data['Text']
y = data['Label']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

# Analyze sentiment using TextBlob pretrained model
def analyze_sentiment(sentence):
    blob = TextBlob(sentence)
    polarity = blob.sentiment.polarity

    # Classify the sentiment based on the polarity score
    if polarity > 0:
        return 1  # Positive
    elif polarity < 0:
        return 0  # Negative
    else:
        return 2  # Neutral

# Apply sentiment analysis to validation set
y_val_predicted = x_val.apply(analyze_sentiment)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_val_predicted)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5600060231892787


In [9]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
X_val_tfidf = tfidf_vectorizer.transform(x_val)
print(data)

                                                    Text Language  Label
3      rwanda set host headquarters united nation dev...       en      1
5      suck since im focused nature aspect thing envi...       en      0
7      shawntarloff itsmieu also relate art lot peopl...       en      2
8      social security constant political crisis dist...       en      0
9      filmthepolicela broken rib puncture lung lead ...       en      0
...                                                  ...      ...    ...
29995            okay getting somewhere httpstcozjinamlr       en      2
29996    yall dont understand excited httpstcolpvogioidq       en      1
29997  talking im saddened today justice wasnt done y...       en      0
29998  mushkamushk lisamwake crashing party long ago ...       en      1
29999  theogouldart tonipayne one beautiful benefit t...       en      1

[22134 rows x 3 columns]


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
logistic_regression_model = LogisticRegression(max_iter=500)
logistic_regression_model.fit(X_train_tfidf, y_train)

train_preds = logistic_regression_model.predict(X_train_tfidf)
val_preds = logistic_regression_model.predict(X_val_tfidf)

train_accuracy = accuracy_score(y_train, train_preds)
val_accuracy = accuracy_score(y_val, val_preds)

print("Train Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)
print("\nTrain Classification Report:")
print(classification_report(y_train, train_preds))
print("\nValidation Classification Report:")
print(classification_report(y_val, val_preds))

Train Accuracy: 0.9776027883560318
Validation Accuracy: 0.9593434723686192

Train Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5533
           1       0.98      0.98      0.98      5551
           2       0.97      0.97      0.97      4409

    accuracy                           0.98     15493
   macro avg       0.98      0.98      0.98     15493
weighted avg       0.98      0.98      0.98     15493


Validation Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2345
           1       0.96      0.97      0.97      2469
           2       0.96      0.94      0.95      1827

    accuracy                           0.96      6641
   macro avg       0.96      0.96      0.96      6641
weighted avg       0.96      0.96      0.96      6641



In [11]:
import random
num_samples = 5
random_indices = random.sample(range(len(data)), num_samples)

for idx in random_indices:
    text_sample = data['Text'].iloc[idx]
    actual_label = data['Label'].iloc[idx]
    text_sample_tfidf = tfidf_vectorizer.transform([text_sample])
    prediction = logistic_regression_model.predict(text_sample_tfidf)

    print("Chosen Text:", text_sample)
    print("Actual Label:", actual_label)
    print("Predicted Label:", prediction)
    print()

Chosen Text: amount time got touched nasty strange men today bc ur drunk im pretty waitress doesnt mean u grab ask nonsensical question drunk guy omw home ill bite ur head
Actual Label: 0
Predicted Label: [0]

Chosen Text: cheek flushed red right side seems purple bruise blush probably hit took fron shitty manager earlier mention taehyungs soft pink lip busted dried blood side
Actual Label: 2
Predicted Label: [2]

Chosen Text: dangerangel schildofgod kellyminor natepickowicz must christian seem poor understanding topic
Actual Label: 0
Predicted Label: [0]

Chosen Text: death mother conversation sibling little brother said doesnt want live heretembisa anymore there much memory mom dad want go live somewhere start afresh life
Actual Label: 2
Predicted Label: [2]

Chosen Text: ama artpatronamar rwdcollective amp givenchy one best ive heard term utility pride holder access exclusive event sound incredible perk hopefully brand follow suit veve ecomi
Actual Label: 1
Predicted Label: [1]



In [12]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_features=1000) 
X_train_counts = count_vectorizer.fit_transform(x_train)
X_val_counts = count_vectorizer.transform(x_val)

svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train_counts, y_train)
train_accuracy = svm_model.score(X_train_counts, y_train)
val_accuracy = svm_model.score(X_val_counts, y_val)

print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)
y_pred = svm_model.predict(X_val_counts)
print(classification_report(y_val, y_pred))

Training Accuracy: 0.9893500290453753
Validation Accuracy: 0.9540731817497364
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2345
           1       0.97      0.96      0.96      2469
           2       0.95      0.94      0.94      1827

    accuracy                           0.95      6641
   macro avg       0.95      0.95      0.95      6641
weighted avg       0.95      0.95      0.95      6641



In [13]:
import random
num_samples = 5
random_indices = random.sample(range(len(data)), num_samples)
for idx in random_indices:
    text_sample = data['Text'].iloc[idx]
    actual_label = data['Label'].iloc[idx]
    text_sample_counts = count_vectorizer.transform([text_sample])
    prediction = svm_model.predict(text_sample_counts)

    print("Chosen Text:", text_sample)
    print("Actual Label:", actual_label)
    print("Predicted Label:", prediction)
    print()

Chosen Text: freedomfight atzoomrr jackklompus nypost law abiding already suffering law abiding almost every horrific mass shooting come legally obtained weapon even law abiding part flawed dont know someone insane something crazy lock gun
Actual Label: 0
Predicted Label: [0]

Chosen Text: need like people skilled color grading look everything let know perfect else literally stress one photo hour
Actual Label: 1
Predicted Label: [1]

Chosen Text: get kid outdoors new environmentally friendly experience new magical park pindara reserve langwarrin find httpstcosokyozlpl frankstoncity kid activity outdoors innovation mediarelease httpstcovjuox
Actual Label: 1
Predicted Label: [1]

Chosen Text: shittyon maybe brazilian portuguese since one known place sugar harvested
Actual Label: 2
Predicted Label: [2]

Chosen Text: woman dont know shell find dream
Actual Label: 1
Predicted Label: [1]



In [19]:
data['tweet_len'] = [len(text.split()) for text in data.Text] 
data = data[~(data['tweet_len'] < 5) & ~(data['tweet_len'] > 60)]
print(data)
data_labels = {key : value for value , key in enumerate(np.unique(data['Label']))}
data_labels
def getlabel(n) : 
    for x , y in data_labels.items() : 
        if y==n : 
            return x
MAXLEN = np.max(data['tweet_len'])
labels = lb.fit_transform(data['Label'].values)
np.unique(labels)
vec = TfidfVectorizer(strip_accents='ascii',analyzer='word',smooth_idf=True, norm=None, min_df=10)
vec.fit(data['Text'])
Features = vec.fit_transform(data['Text']).toarray()
X_train , X_val , Y_train , Y_val = train_test_split(Features , labels , train_size = 0.8 , random_state = 42)

                                                    Text Language  Label  \
3      rwanda set host headquarters united nation dev...       en      1   
5      suck since im focused nature aspect thing envi...       en      0   
7      shawntarloff itsmieu also relate art lot peopl...       en      2   
8      social security constant political crisis dist...       en      0   
9      filmthepolicela broken rib puncture lung lead ...       en      0   
...                                                  ...      ...    ...   
29994  match history broke lost exec team wailords gg...       en      0   
29996    yall dont understand excited httpstcolpvogioidq       en      1   
29997  talking im saddened today justice wasnt done y...       en      0   
29998  mushkamushk lisamwake crashing party long ago ...       en      1   
29999  theogouldart tonipayne one beautiful benefit t...       en      1   

       tweet_len  
3             19  
5             27  
7             27  
8          

In [20]:
def Vectorization(col , MAXLEN = 60) : 
    sents = data[col].tolist() 
    
    # build vocabulary 
    corpus = [word for text in data[col] for word in text.split()] 
    vocab_size = len(Counter(corpus)) 
    
    tokenizer = Tokenizer(num_words=vocab_size , oov_token = "<OOV>" , 
                          filters='!#$%&()*+,-/:;<=>@«»""[\\]^_`{|}~\t\n'
                         )
    
    tokenizer.fit_on_texts(sents) 
    
    tokenizer.word_index['<pad>'] = 0 
    tokenizer.index_word[0] = '<pad>' 
    
    # Text Vectorization 
    seqs = tokenizer.texts_to_sequences(sents) 
    
    pad_seqs = pad_sequences(seqs , maxlen = MAXLEN , padding = 'post') 
    
    return pad_seqs , tokenizer

seqs , tokenizer = Vectorization('Text')
seq_sent = ' '.join([tokenizer.index_word[idx] for idx in seqs[10] if idx != 0])  
print(f"{seqs[10]} \n\n {seq_sent}")

# define parameters 
VOCAB_SIZE = len(tokenizer.word_index) 
EMBEDDING_DIM = 256
#MAXLEN = np.max(data['tweet_len'])
#word2vec
Word2vec_train_data = list(map(lambda x:x.split() , data['Text'].tolist()))
word2vec_model = Word2Vec(Word2vec_train_data , vector_size = EMBEDDING_DIM)
print("Vocabulary Length:", len(word2vec_model.wv.key_to_index))
embedding_matrix = np.zeros((VOCAB_SIZE , EMBEDDING_DIM)) 
for word , token in tokenizer.word_index.items() : 
    if word2vec_model.wv.__contains__(word) : 
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)
        
print("Embedding Matrix Shape:", embedding_matrix.shape)
seqs_labels = tf.keras.utils.to_categorical(labels , len(np.unique(labels)))

[  236    65   954  7936 10247   475  1546   198  1595   193  3313   748
    43 15579 15580   856 15581 15582     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0] 

 power back north vi ladysmith near block th ave cause motor vehicle accident bchydro britishcolumbia canada poweroutage poweron
Vocabulary Length: 6575
Embedding Matrix Shape: (59073, 256)


In [21]:
x_train , x_val , y_train , y_val = train_test_split(seqs , seqs_labels , train_size = 0.80, random_state = 42)
x_train.shape , x_val.shape , y_train.shape , y_val.shape
BATCH_SIZE = 128
BUFFER_SIZE = 1000
train_set = tf.data.Dataset.from_tensor_slices((x_train , y_train))
train_set = train_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE , drop_remainder = True)

val_set = tf.data.Dataset.from_tensor_slices((x_val , y_val))
val_set = val_set.batch(BATCH_SIZE , drop_remainder = True)

In [22]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model = Sequential([
    Embedding(
        input_dim = VOCAB_SIZE , 
        output_dim= EMBEDDING_DIM , 
        weights = [embedding_matrix] , 
        input_length = MAXLEN ,
        trainable=False
    ) , 
    Bidirectional(LSTM(256 , return_sequences=True)) , 
    Bidirectional(LSTM(128)) , 
    Dense(64 , activation='relu') , 
    Dropout(0.2) , 
    Dense(3 , activation='softmax')
])
#train our model
model.compile(optimizer=Adam(0.002) , loss='categorical_crossentropy' , metrics=['accuracy', Precision(name = 'precision'), Recall(name = 'recall')])
history = model.fit(
    train_set, 
    epochs=40, 
    validation_data=val_set, 
    callbacks=[early_stopping]
)


Epoch 1/40


125/125 [==============================] - 35s 252ms/step - loss: 1.0471 - accuracy: 0.4544 - precision: 0.5916 - recall: 0.1328 - val_loss: 1.0080 - val_accuracy: 0.4796 - val_precision: 0.7817 - val_recall: 0.1678
Epoch 2/40
125/125 [==============================] - 30s 243ms/step - loss: 0.8944 - accuracy: 0.5787 - precision: 0.6853 - recall: 0.3928 - val_loss: 0.7607 - val_accuracy: 0.6436 - val_precision: 0.7583 - val_recall: 0.5101
Epoch 3/40
125/125 [==============================] - 34s 273ms/step - loss: 0.6132 - accuracy: 0.7413 - precision: 0.7910 - recall: 0.6666 - val_loss: 0.4157 - val_accuracy: 0.8397 - val_precision: 0.8481 - val_recall: 0.8261
Epoch 4/40
125/125 [==============================] - 32s 252ms/step - loss: 0.3406 - accuracy: 0.8781 - precision: 0.8875 - recall: 0.8665 - val_loss: 0.2598 - val_accuracy: 0.9057 - val_precision: 0.9093 - val_recall: 0.8992
Epoch 5/40
125/125 [==============================] - 34s 274ms/step - loss: 0.2050 - acc

In [25]:
#prediction
def make_predictions(row) : 
    random_data = row.sample(n = 30) 
    random_tweets = random_data['Text'].values 
    y_test = random_data['Label'].values
    
    x_test = pad_sequences(tokenizer.texts_to_sequences(random_tweets) , maxlen=MAXLEN , padding = 'post')
    
    test_preds = model.predict(x_test).round() 
    
    for i in tqdm(range(10)) : 
        print(f"The original tweet : {random_tweets[i]}\n")
        print(f"The original label : {y_test[i]}\n")
        print(f"The prediction is : {getlabel(test_preds[i].argmax())}\n")
        print('-'*120)
        
make_predictions(data)

1/1 [==============================] - 0s 40ms/step


  0%|          | 0/10 [00:00<?, ?it/s]

The original tweet : poppysdaddi keptadams he wrong stopped

The original label : 0

The prediction is : 0

------------------------------------------------------------------------------------------------------------------------
The original tweet : teach question everything find fact find truth raised valuing science research never stop curious informed shock roe v wade overturned wore got angry scared still

The original label : 0

The prediction is : 0

------------------------------------------------------------------------------------------------------------------------
The original tweet : paullahticks well see problem obviously little tiny hand cant fit around grown man neck httpstcohaijrqg

The original label : 0

The prediction is : 0

------------------------------------------------------------------------------------------------------------------------
The original tweet : kidsmentalhlth kmoranont alot young black male extreme mental issue daily stuggles afraid coming maybe 

In [26]:
#gui 
import tkinter as tk
from tkinter import ttk
from ttkthemes import ThemedStyle
from keras.preprocessing.sequence import pad_sequences
import numpy as np

class SentimentAnalysisApp:
    def __init__(self, model, tokenizer, maxlen):
        self.model = model
        self.tokenizer = tokenizer
        self.maxlen = maxlen
        
        self.window = tk.Tk()
        self.window.title("Sentiment Analysis App")
        
        # Apply themed style
        self.style = ThemedStyle(self.window)
        self.style.theme_use('arc')  # Choose theme ('clearlooks', 'kroc', 'plastik', 'radiance', etc.)

        self.create_widgets()
    
    def create_widgets(self):
        # Create input label
        input_label = ttk.Label(self.window, text="Enter Text:", foreground='blue', font=('Arial', 14, 'bold'))
        input_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

        # Create input entry
        self.input_text = ttk.Entry(self.window, width=50, font=('Arial', 12))
        self.input_text.grid(row=0, column=1, padx=10, pady=10)

        # Create predict button
        self.predict_button = ttk.Button(self.window, text="Predict", command=self.predict, style='Accent.TButton')
        self.predict_button.grid(row=0, column=2, padx=10, pady=10)

        # Create output label
        self.output_label = ttk.Label(self.window, text="", font=('Arial', 14, 'bold'))
        self.output_label.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

    def predict(self):
        text = self.input_text.get()
        prediction = self.predict_sentiment(text)
        self.output_label.config(text=f"Prediction: {prediction}", foreground=self.get_label_color(prediction))

    def predict_sentiment(self, text):
        # Tokenize the input text
        tokenized_text = self.tokenizer.texts_to_sequences([text])
        # Pad sequences to fixed length
        padded_text = pad_sequences(tokenized_text, maxlen=self.maxlen)
        # Make prediction
        prediction = self.model.predict(padded_text)
        # Map prediction to sentiment label
        sentiment_label = np.argmax(prediction)
        if sentiment_label == 0:
            return "Negative"
        elif sentiment_label == 1:
            return "Positive"
        else:
            return "Neutral"

    def get_label_color(self, prediction):
        if prediction == "Positive":
            return "green"
        elif prediction == "Negative":
            return "red"
        else:
            return "blue"

if __name__ == "__main__":
    # Replace 'model', 'tokenizer', and 'MAXLEN' with your actual model, tokenizer, and maxlen
    app = SentimentAnalysisApp(model, tokenizer, MAXLEN)
    app.window.mainloop()